# USING AdaNet on MNIST Dataset

In [ ]:
!pip install adanet

In [ ]:
!pip install tensorflow

In [54]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools

import adanet
from adanet.examples import simple_dnn
import tensorflow as tf

# The random seed to use.
RANDOM_SEED = 42

In [6]:
(x_train, y_train), (x_test, y_test) = (
    tf.keras.datasets.mnist.load_data())

11493376/11490434 [==============================] - 16s 1us/step


In [21]:
FEATURES_KEY = "images"


def generator(images, labels):
  """Returns a generator that returns image-label pairs."""

  def _gen():
    for image, label in zip(images, labels):
      yield image, label

  return _gen


def preprocess_image(image, label):
  """Preprocesses an image for an `Estimator`."""
  # Pixel degerleri 0 ve 1 arasinda olmali.
  image = image / 255.
  # 2D-convolution uygulamak icin resimleri yeniden bicimlendirmeliyiz.
  image = tf.reshape(image, [28, 28, 1])
  # Ozelliklerimizi bir dictonary objesinde toparlamaliyiz.
  features = {FEATURES_KEY: image}
  return features, label


def input_fn(partition, training, batch_size):
  """Generate an input_fn for the Estimator."""

  def _input_fn():
    if partition == "train":
      dataset = tf.data.Dataset.from_generator(
          generator(x_train, y_train), (tf.float32, tf.int32), ((28, 28), ()))
    else:
      dataset = tf.data.Dataset.from_generator(
          generator(x_test, y_test), (tf.float32, tf.int32), ((28, 28), ()))

    
    if training:
      dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

    dataset = dataset.map(preprocess_image).batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

  return _input_fn

In [22]:
# Siniflandirma ogelerinin sayisi.
NUM_CLASSES = 10

# Gradyanlar hesaplanirken her mini-batch uzerinde ortalama loss hesaplanacak.
loss_reduction = tf.losses.Reduction.SUM_OVER_BATCH_SIZE

# `Head`, `Estimator`lar icin loss function ve metrikleri tanimliyor.
head = tf.contrib.estimator.multi_class_head(
    NUM_CLASSES, loss_reduction=loss_reduction)

# Bazi `Estimator`lar girdi ozelliklerini anlamak icin ozellik sutunlari kullaniyor.
feature_columns = [
    tf.feature_column.numeric_column(FEATURES_KEY, shape=[28, 28, 1])
]

# Estimator konfigurasyonu.
config = tf.estimator.RunConfig(
    save_checkpoints_steps=50000,
    save_summary_steps=50000,
    tf_random_seed=RANDOM_SEED)

In [9]:
# Once basit bir lineer model egitelim
LEARNING_RATE = 0.001  
TRAIN_STEPS = 5000  
BATCH_SIZE = 64  

estimator = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=NUM_CLASSES,
    optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
    loss_reduction=loss_reduction,
    config=config)

results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None))
print("Accuracy:", results["accuracy"])
print("Loss:", results["average_loss"])

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\dell\\AppData\\Local\\Temp\\tmp2ktdva5_', '_tf_random_seed': 42, '_save_summary_steps': 50000, '_save_checkpoints_steps': 50000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000243B40E8E48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflo

global step 5000: accuracy = 0.9258, average_loss = 0.27285188, global_step = 5000, loss = 0.27215475
Loss for final step: 0.13550904.
Accuracy: 0.9258
Loss: 0.27285188

Varsayilan degerler kullanarak olusturulan lineer model yaklasik %92.58 dogruluk degeri elde etti.

Bakalim AdaNet simple_dnn modelini kullanarak daha iyi bir sonuc elde edebilecek miyiz?

In [10]:
LEARNING_RATE = 0.003  
TRAIN_STEPS = 5000  
BATCH_SIZE = 64  
ADANET_ITERATIONS = 2  

estimator = adanet.Estimator(
    head=head,
    subnetwork_generator=simple_dnn.Generator(
        feature_columns=feature_columns,
        optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
        seed=RANDOM_SEED),
    max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,
    evaluator=adanet.Evaluator(
        input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE),
        steps=None),
    config=config)

results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None))
print("Accuracy:", results["accuracy"])
print("Loss:", results["average_loss"])

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\dell\\AppData\\Local\\Temp\\tmp675d5x3w', '_tf_random_seed': 42, '_save_summary_steps': 50000, '_save_checkpoints_steps': 50000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000243B4452E48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflo

INFO:tensorflow:Finished training Adanet iteration 0
INFO:tensorflow:Beginning training AdaNet iteration 1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Importing frozen ensemble from C:\Users\dell\AppData\Local\Temp\tmp675d5x3w\frozen/ensemble-0.meta with features: ['images'].
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\dell\AppData\Local\Temp\tmp675d5x3w\increment.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2500 into C:\Users\dell\AppData\Local\Temp\tmp675d5x3w\model.ckpt.
INFO:tensorflow:loss = 0.10201884, step = 2501
INFO:tensorflow:global_step/sec: 63.9774
INFO:tensorflow:loss = 0.047877535, step = 2601 (1.563 sec)
INFO:tensorflow:global_step/sec: 91.6315
INFO:tensorflow:loss = 0.068694524, step = 2701 (1.091 sec)
INFO:tensorflow:global_step/sec: 91.0478
INFO:tensorflo

Saving dict for global step 5000: accuracy = 0.957, accuracy/adanet/adanet_weighted_ensemble = 0.957, accuracy/adanet/subnetwork = 0.957, accuracy/adanet/uniform_average_ensemble = 0.957, average_loss = 0.14346279, average_loss/adanet/adanet_weighted_ensemble = 0.14346279, average_loss/adanet/subnetwork = 0.14346279, average_loss/adanet/uniform_average_ensemble = 0.14346279, global_step = 5000, loss = 0.14280926, loss/adanet/adanet_weighted_ensemble = 0.14280926, loss/adanet/subnetwork = 0.14280926, loss/adanet/uniform_average_ensemble = 0.14280926
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\dell\AppData\Local\Temp\tmp675d5x3w\model.ckpt-5000
INFO:tensorflow:Loss for final step: 0.029671986.
Accuracy: 0.957
Loss: 0.14346279

AdaNet'in simple_dnn modeli varsayilan degerlerle basarimi %95.7'ye cikardi.

In [51]:
class SimpleCNNBuilder(adanet.subnetwork.Builder):
  """Builds a CNN subnetwork for AdaNet."""

  def __init__(self, learning_rate, max_iteration_steps, seed):
    """Initializes a `SimpleCNNBuilder`.

    Args:
      learning_rate: The float learning rate to use.
      max_iteration_steps: The number of steps per iteration.
      seed: The random seed.

    Returns:
      An instance of `SimpleCNNBuilder`.
    """
    self._learning_rate = learning_rate
    self._max_iteration_steps = max_iteration_steps
    self._seed = seed

  def build_subnetwork(self,
                       features,
                       logits_dimension,
                       training,
                       iteration_step,
                       summary,
                       previous_ensemble=None):
    """See `adanet.subnetwork.Builder`."""
    images = tf.to_float(features[FEATURES_KEY])
    kernel_initializer = tf.keras.initializers.he_normal(seed=self._seed)
    x = tf.layers.conv2d(
        images,
        filters=16,
        kernel_size=3,
        padding="same",
        activation="relu",
        kernel_initializer=kernel_initializer)
    x = tf.layers.max_pooling2d(x, pool_size=2, strides=2)
    x = tf.layers.flatten(x)
    x = tf.layers.dense(
        x, units=64, activation="relu", kernel_initializer=kernel_initializer)

    # The `Head` passed to adanet.Estimator will apply the softmax activation.
    logits = tf.layers.dense(
        x, units=10, activation=None, kernel_initializer=kernel_initializer)

    # Use a constant complexity measure, since all subnetworks have the same
    # architecture and hyperparameters.
    complexity = tf.constant(1)

    return adanet.Subnetwork(
        last_layer=x,
        logits=logits,
        complexity=complexity,
        persisted_tensors={})

  def build_subnetwork_train_op(self, 
                                subnetwork, 
                                loss, 
                                var_list, 
                                labels, 
                                iteration_step,
                                summary, 
                                previous_ensemble=None):
    """See `adanet.subnetwork.Builder`."""

    # Momentum optimizer with cosine learning rate decay works well with CNNs.
    learning_rate = tf.train.cosine_decay(
        learning_rate=self._learning_rate,
        global_step=iteration_step,
        decay_steps=self._max_iteration_steps)
    optimizer = tf.train.MomentumOptimizer(learning_rate, .9)
    # NOTE: The `adanet.Estimator` increments the global step.
    return optimizer.minimize(loss=loss, var_list=var_list)

  def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                     iteration_step, summary):
    """See `adanet.subnetwork.Builder`."""
    return tf.no_op("mixture_weights_train_op")

  @property
  def name(self):
    """See `adanet.subnetwork.Builder`."""
    return "simple_cnn"

In [52]:
class SimpleCNNGenerator(adanet.subnetwork.Generator):
  """Generates a `SimpleCNN` at each iteration.
  """

  def __init__(self, learning_rate, max_iteration_steps, seed=None):
    """Initializes a `Generator` that builds `SimpleCNNs`.

    Args:
      learning_rate: The float learning rate to use.
      max_iteration_steps: The number of steps per iteration.
      seed: The random seed.

    Returns:
      An instance of `Generator`.
    """
    self._seed = seed
    self._dnn_builder_fn = functools.partial(
        SimpleCNNBuilder,
        learning_rate=learning_rate,
        max_iteration_steps=max_iteration_steps)

  def generate_candidates(self, previous_ensemble, iteration_number,
                          previous_ensemble_reports, all_reports):
    """See `adanet.subnetwork.Generator`."""
    seed = self._seed
    # Change the seed according to the iteration so that each subnetwork
    # learns something different.
    if seed is not None:
      seed += iteration_number
    return [self._dnn_builder_fn(seed=seed)]

In [53]:
LEARNING_RATE = 0.05  #@param {type:"number"}
TRAIN_STEPS = 5000  #@param {type:"integer"}
BATCH_SIZE = 64  #@param {type:"integer"}
ADANET_ITERATIONS = 2  #@param {type:"integer"}

max_iteration_steps = TRAIN_STEPS // ADANET_ITERATIONS
estimator = adanet.Estimator(
    head=head,
    subnetwork_generator=SimpleCNNGenerator(
        learning_rate=LEARNING_RATE,
        max_iteration_steps=max_iteration_steps,
        seed=RANDOM_SEED),
    max_iteration_steps=max_iteration_steps,
    evaluator=adanet.Evaluator(
        input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE),
        steps=None),
    report_materializer=adanet.ReportMaterializer(
        input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE),
        steps=None),
    adanet_loss_decay=.99,
    config=config)

results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None))
print("Accuracy:", results["accuracy"])
print("Loss:", results["average_loss"])

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\dell\\AppData\\Local\\Temp\\tmp3rlexm4i', '_tf_random_seed': 42, '_save_summary_steps': 50000, '_save_checkpoints_steps': 50000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000243BCA3C4A8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflo

INFO:tensorflow:loss = 0.02847917, step = 2501
INFO:tensorflow:global_step/sec: 23.8725
INFO:tensorflow:loss = 0.004389563, step = 2601 (4.190 sec)
INFO:tensorflow:global_step/sec: 26.4853
INFO:tensorflow:loss = 0.0059942594, step = 2701 (3.776 sec)
INFO:tensorflow:global_step/sec: 25.4221
INFO:tensorflow:loss = 0.080152735, step = 2801 (3.975 sec)
INFO:tensorflow:global_step/sec: 26.6401
INFO:tensorflow:loss = 0.008667019, step = 2901 (3.713 sec)
INFO:tensorflow:global_step/sec: 25.9765
INFO:tensorflow:loss = 0.042215403, step = 3001 (3.850 sec)
INFO:tensorflow:global_step/sec: 26.4994
INFO:tensorflow:loss = 0.0013359454, step = 3101 (3.774 sec)
INFO:tensorflow:global_step/sec: 27.4066
INFO:tensorflow:loss = 0.0086673405, step = 3201 (3.649 sec)
INFO:tensorflow:global_step/sec: 26.598
INFO:tensorflow:loss = 0.004777601, step = 3301 (3.760 sec)
INFO:tensorflow:global_step/sec: 24.7419
INFO:tensorflow:loss = 0.0063926866, step = 3401 (4.042 sec)
INFO:tensorflow:global_step/sec: 27.2555


Saving dict for global step 5000: accuracy = 0.9869, accuracy/adanet/adanet_weighted_ensemble = 0.9869, accuracy/adanet/subnetwork = 0.9856, accuracy/adanet/uniform_average_ensemble = 0.9869, average_loss = 0.03953884, average_loss/adanet/adanet_weighted_ensemble = 0.03953884, average_loss/adanet/subnetwork = 0.04450064, average_loss/adanet/uniform_average_ensemble = 0.03953884, global_step = 5000, loss = 0.03935165, loss/adanet/adanet_weighted_ensemble = 0.03935165, loss/adanet/subnetwork = 0.044292618, loss/adanet/uniform_average_ensemble = 0.03935165
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\dell\AppData\Local\Temp\tmp3rlexm4i\model.ckpt-5000
INFO:tensorflow:Loss for final step: 0.02186182.
Accuracy: 0.9869
Loss: 0.03953884

Ve MNIST icin %98.69 basarim elde etmis olduk.